In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import pickle

In [16]:
# Load Data
data = pd.read_csv('/Users/sauce/Desktop/DraftKings/ready_data/Mock_rest2.csv', header = None)



# rename columns
data = data.rename(columns={0: "season", 
                        1:'game_date',
                        2: 'player',
                        3: 'position',    
                        4: 'team',
                        5: 'opponent',
                        6: 'venue',
                        7: 'minutes',
                        8: 'usage_rate',
                        9: 'rest',
                        10: 'avg_threes',
                        11: 'avg_reb',
                        12: 'avg_ast',
                        13: 'avg_stl',
                        14: 'avg_blk',
                        15: 'avg_tov',
                        16: 'avg_pts',
                        17: 'avg_points_vs_opp',
                        18: 'team_pace',
                        19: 'team_ast',
                        20: 'team_tov',
                        21: 'team_reb_rate',
                        22: 'team_offeff',
                        23: 'team_defeff',
                        24: 'opp_pace',
                        25: 'opp_ast',
                        26: 'opp_tov',
                        27: 'opp_reb_rate',
                        28: 'opp_offeff',
                        29: 'opp_defeff',
                        30: 'opp_pos_avg',
                        31: 'salary',
                        32: 'fantasy_points',
                       })


# Avg 10 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(10).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
data = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG10'])

# Avg 3 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(3).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
df = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG3'])

# Drop useless features
df = df.drop(columns=['team_AVG10','opponent_AVG10','venue_AVG10','rest_AVG10',
                      'avg_threes_AVG10','avg_reb_AVG10','avg_ast_AVG10','avg_stl_AVG10',
                      'avg_blk_AVG10','avg_tov_AVG10','avg_pts_AVG10','avg_points_vs_opp_AVG10',
                      'team_pace_AVG10','team_ast_AVG10','team_tov_AVG10','team_reb_rate_AVG10',
                      'team_defeff_AVG10','opp_pace_AVG10','opp_ast_AVG10','opp_tov_AVG10',
                      'opp_reb_rate_AVG10','opp_offeff_AVG10','salary_AVG10',
                      'team_AVG3', 'opponent_AVG3', 'venue_AVG3','rest_AVG3',
                      'avg_threes_AVG3', 'avg_reb_AVG3','avg_ast_AVG3', 'avg_stl_AVG3', 
                      'avg_blk_AVG3', 'avg_tov_AVG3','avg_pts_AVG3', 
                      'avg_points_vs_opp_AVG3', 'team_pace_AVG3','team_ast_AVG3', 
                      'team_tov_AVG3', 'team_reb_rate_AVG3','team_defeff_AVG3', 
                      'opp_pace_AVG3','opp_ast_AVG3', 'opp_tov_AVG3', 'opp_reb_rate_AVG3',
                      'opp_offeff_AVG3', 'opp_pos_avg_AVG3','salary_AVG3',
                      'team_AVG10_AVG3','opponent_AVG10_AVG3', 'venue_AVG10_AVG3', 
                      'minutes_AVG10_AVG3','usage_rate_AVG10_AVG3', 'rest_AVG10_AVG3',
                      'avg_threes_AVG10_AVG3', 'avg_reb_AVG10_AVG3',
                      'avg_ast_AVG10_AVG3', 'avg_stl_AVG10_AVG3', 'avg_blk_AVG10_AVG3',
                      'avg_tov_AVG10_AVG3', 'avg_pts_AVG10_AVG3',
                      'avg_points_vs_opp_AVG10_AVG3', 'team_pace_AVG10_AVG3',
                      'team_ast_AVG10_AVG3', 'team_tov_AVG10_AVG3',
                      'team_reb_rate_AVG10_AVG3', 'team_offeff_AVG10_AVG3',
                      'team_defeff_AVG10_AVG3', 'opp_pace_AVG10_AVG3',
                      'opp_ast_AVG10_AVG3', 'opp_tov_AVG10_AVG3',
                      'opp_reb_rate_AVG10_AVG3', 'opp_offeff_AVG10_AVG3',
                      'opp_defeff_AVG10_AVG3', 'opp_pos_avg_AVG10_AVG3',
                      'salary_AVG10_AVG3', 'fantasy_points_AVG10_AVG3', 'position_AVG10_AVG3', 'position_AVG3', 'position_AVG10'])

# Fill NAs
df['fantasy_points_AVG3'] = df['fantasy_points_AVG3'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['opp_defeff_AVG3'] = df['opp_defeff_AVG3'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG3'] = df['team_offeff_AVG3'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG3'] = df['usage_rate_AVG3'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['fantasy_points_AVG10'] = df['fantasy_points_AVG10'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['minutes_AVG3'] = df['minutes_AVG3'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))
df['opp_pos_avg_AVG10'] = df['opp_pos_avg_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_pos_avg'].transform('mean'))
df['opp_defeff_AVG10'] = df['opp_defeff_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG10'] = df['team_offeff_AVG10'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG10'] = df['usage_rate_AVG10'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['minutes_AVG10'] = df['minutes_AVG10'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))
 


df.to_csv('/Users/sauce/Desktop/DraftKings/ready_data/mock_rest.csv',
                        header=True,
                        index=False,
                        sep=',')

In [15]:
df

,game_date,season,player,position,team,opponent,venue,minutes,usage_rate,rest,...,usage_rate_AVG10,team_offeff_AVG10,opp_defeff_AVG10,opp_pos_avg_AVG10,fantasy_points_AVG10,minutes_AVG3,usage_rate_AVG3,team_offeff_AVG3,opp_defeff_AVG3,fantasy_points_AVG3
0,2016-10-26,﻿2016-2017 Regular Season,Mock0,SG,Orlando,Miami,H,39.38,25.05,3+,...,25.05,101.2,108.1,19.13,35.50,39.38,25.05,101.2,108.1,35.50
1,2016-10-28,2016-2017 Regular Season,Mock1,PF/C,Indiana,Brooklyn,R,32.90,16.22,1,...,16.22,106.2,106.3,25.05,33.75,32.90,16.22,106.2,106.3,33.75
2,2016-10-28,2016-2017 Regular Season,Mock2,PG,Golden State,New Orleans,R,34.07,29.73,2,...,29.73,113.2,101.1,23.49,38.25,34.07,29.73,113.2,101.1,38.25
3,2016-10-29,2016-2017 Regular Season,Mock3,SF,Cleveland,Orlando,H,23.12,13.19,B2B,...,13.19,110.9,108.0,18.76,11.25,23.12,13.19,110.9,108.0,11.25
4,2016-11-02,2016-2017 Regular Season,Mock5,SF,Houston,New York,R,18.85,16.60,3IN4-B2B,...,16.60,111.8,106.4,18.72,16.25,18.85,16.60,111.8,106.4,16.25
8,2016-11-21,2016-2017 Regular Season,Mock9,PG,LA Clippers,Toronto,H,15.87,18.12,3IN4,...,18.12,110.3,105.8,22.50,5.25,15.87,18.12,110.3,105.8,5.25
5,2016-12-10,2016-2017 Regular Season,Mock6,PG,Miami,Chicago,R,34.58,30.58,4IN5-B2B,...,30.58,105.2,104.1,23.39,42.75,34.58,30.58,105.2,104.1,42.75
7,2018-03-22,2017-2018 Regular Season,Mock8,PF/C,New Orleans,LA Lakers,H,36.57,28.42,5IN5-B2B2B,...,28.42,107.7,105.6,21.82,55.75,36.57,28.42,107.7,105.6,55.75
6,2018-03-24,2017-2018 Regular Season,Mock7,SG,New Orleans,Houston,R,6.55,12.50,4IN5,...,12.50,107.7,105.6,15.86,1.25,6.55,12.50,107.7,105.6,1.25
